In [5]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pathlib
# from data import get_mnist
from pathlib import Path
from array import array
import struct



In [6]:
def read_image_data(path):
    data_dir = Path("./data")
    with open(data_dir / path, "rb") as f:
        # IDX file format
        magic, size, rows, cols = struct.unpack(">IIII", f.read(16))
        image_data = array("B", f.read())   
    images = []
    for i in range(size):
        image = np.array(image_data[i * rows * cols:(i + 1) * rows * cols]).reshape(28, 28)
        images.append(image)
    return np.array(images)


def read_labels(path):
    data_dir = Path("./data")
    with open(data_dir / path, "rb") as f:
        magic, size = struct.unpack(">II", f.read(8))
        if magic != 2049:
            raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
        labels = np.array(array("B", f.read()))
    return labels

In [7]:
def read_image_data(path):
    data_dir = Path("./data")
    with open(data_dir / path, "rb") as f:
        # IDX file format
        magic, size, rows, cols = struct.unpack(">IIII", f.read(16))
        image_data = array("B", f.read())   
    images = []
    for i in range(size):
        image = np.array(image_data[i * rows * cols:(i + 1) * rows * cols]).reshape(28, 28)
        images.append(image)
    return np.array(images)


def read_labels(path):
    data_dir = Path("./data")
    with open(data_dir / path, "rb") as f:
        magic, size = struct.unpack(">II", f.read(8))
        if magic != 2049:
            raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
        labels = np.array(array("B", f.read()))
    return labels

In [8]:
train_labels = read_labels("train-labels.idx1-ubyte")
train_images = read_image_data("train-images.idx3-ubyte")
test_images = read_image_data("t10k-images.idx3-ubyte")
test_labels = read_labels("t10k-labels.idx1-ubyte")

In [9]:
def onehot(label):
    encoded_label = np.zeros(10)
    encoded_label[label] = 1
    return encoded_label

In [22]:
images = train_images.reshape(-1, 28 * 28)
labels = np.array([onehot(label) for label in train_labels])


In [14]:
w_i_h = np.random.uniform(-0.5, 0.5, (20, 784))
w_h_o = np.random.uniform(-0.5, 0.5, (10, 20))
b_i_h = np.zeros((20, 1))
b_h_o = np.zeros((10, 1))

In [23]:
print(images.shape)
print(labels.shape)

(60000, 784)
(60000, 10)


In [24]:
learn_rate = 0.01
nr_correct = 0
epochs = 3

In [25]:
for epoch in range(epochs):
    for img, l in zip(images, labels):
        img.shape += (1,)
        l.shape += (1,)
        # Forward propagation input -> hidden
        h_pre = b_i_h + w_i_h @ img
        h = 1 / (1 + np.exp(-h_pre))
        # Forward propagation hidden -> output
        o_pre = b_h_o + w_h_o @ h
        o = 1 / (1 + np.exp(-o_pre))

        # Cost / Error calculation
        e = 1 / len(o) * np.sum((o - l) ** 2, axis=0)
        nr_correct += int(np.argmax(o) == np.argmax(l))

        # Backpropagation output -> hidden (cost function derivative)
        delta_o = o - l
        
        w_h_o += -learn_rate * delta_o @ np.transpose(h)
        b_h_o += -learn_rate * delta_o
        # Backpropagation hidden -> input (activation function derivative)
        delta_h = np.transpose(w_h_o) @ delta_o * (h * (1 - h))
        w_i_h += -learn_rate * delta_h @ np.transpose(img)
        b_i_h += -learn_rate * delta_h

    # Show accuracy for this epoch
    print(f"Acc: {round((nr_correct / images.shape[0]) * 100, 2)}%")
    nr_correct = 0

/tmp/ipykernel_32256/3999807045.py:7: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-h_pre))


Acc: 29.72%
Acc: 42.53%
Acc: 45.68%


In [ ]:
# Show results
def convert_img_to_mnist(path):
    image = Image.open(path)
    
    # Resize the image to 28x28 pixels
    image = image.resize((28, 28))
    # Convert the image to grayscale
    image = image.convert('L')

    image_array = np.array(image).flatten()

    return image_array 


In [ ]:
image_array = convert_img_to_mnist(f'./img/8.jpg')
h_pre = b_i_h + w_i_h @ image_array.reshape(784, 1)
h = 1 / (1 + np.exp(-h_pre))
o_pre = b_h_o + w_h_o @ h
o = 1 / (1 + np.exp(-o_pre))
print(o.argmax())

3


/tmp/ipykernel_6737/3839041451.py:3: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-h_pre))
